In [ ]:
import matplotlib
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage import feature, io, filters, measure, color, exposure, segmentation, morphology
from skimage.filters import rank
from skimage.morphology import watershed, disk, reconstruction
from scipy.misc import toimage
from scipy import ndimage as ndi
from sklearn.covariance import EllipticEnvelope
from sklearn.externals import joblib
from itertools import compress
import h5py
import datetime
%matplotlib inline

## Functions

In [ ]:
def normalise(inData):
    """
    Normalise array.
    """
    inDataAbs = np.fabs(inData)
    inDataMax = np.amax(inData)
    normalisedData = inDataAbs/inDataMax
    return normalisedData

def cutArr(A):
    """
    Remove rows and columns of zero from input arr A.
    """
    A = A[:,~np.all(A == 0, axis=0)]
    A = A[~np.all(A == 0, axis=1)]
    return A
    
def resizeArray(arr):
    """
    Interpolate array to fit (200,200).
    """

    outArr = np.zeros((200,200))

    # Resize the arr
    ratio = 200.0/np.amax(arr.shape)

    arr = ndi.interpolation.zoom(arr, (ratio))
    outArr[:arr.shape[0],:arr.shape[1]] = arr
    return normalise(outArr)

def genExample(cell):
    cell = cutArr(cell)
    
    vacuoles, mpeaks = getVacuholes(cell)
    vacuoleLabels = measure.label(mpeaks)
        
    return cell, np.ma.masked_array(cell, mpeaks), \
      vacuoleLabels

def getVacuholes(cell):
    """
    Get vacuoles using skimage.morphology.reconstruction()
    """
    seed = np.copy(cell)
    seed[1:-1, 1:-1] = cell.max()
    mask = cell

    filled = reconstruction(seed, mask, method='erosion')

    thresh = filters.threshold_mean(filled-cell)
    mask = (filled-cell) >= thresh
    mask = morphology.remove_small_objects(mask, 100)
    labeled = measure.label(mask)

    vacuoles = ndi.find_objects(labeled)
    
    return np.array(filter(None,vacuoles)), mask

## Find cell locations

The redimg, greenimg, blueimg, uvimg are the filters used in the InCell Analyser 6000. The DAPI and Cy5 channels have been used in the cell/vacuole finding algorithm, and the remaining channels (+ Cy5) have been used to display the results in colour. The DAPI channel uses Hoechst 33342 staining to identify nuclei, and the cell plasma is identified by cell mask staining in the Cy5 channel, with holes in the plasma shown as black spots.

Cells are found using a watershedding method, with nuclei used as initial markers.

In [ ]:
redimg = "./data/H - 11(fld 07 wv Red - Cy5).tif"
greenimg = "./data/H - 11(fld 07 wv Green - dsRed).tif"
blueimg = "./data/H - 11(fld 07 wv Blue - FITC).tif"
uvimg = "./data/H - 11(fld 07 wv UV - DAPI).tif"
# Lower and upper boundaries of inliers, according to genIQR.ipynb:
loBound = 8700
hiBound = 305476

In [ ]:
print("Markers from", uvimg, "Cells from", redimg)

imUV = io.imread(uvimg)
thresh = filters.threshold_li(imUV)
mask = imUV <= thresh
labeled = measure.label(mask, background=1)
markers = rank.median(labeled, disk(25))

imBW = io.imread(redimg)
p0, p1 = np.percentile(imBW, (10, 70)) # These parameters can also be changed
imBWrescaled = exposure.rescale_intensity(imBW, in_range=(p0, p1))
thresh = filters.threshold_li(imBWrescaled)
mask = imBWrescaled <= thresh
gradient = rank.gradient(mask==0, disk(2))

labeled = segmentation.watershed(gradient, markers)
#labeled = segmentation.clear_border(labeled) # Get rid of border cells

cells = filter(None, ndi.find_objects(labeled))

cellImages = []
cellImagesRAW = []
print("Cells found:", len(cells))
if len(cells) != 0:
    for i in np.arange(len(cells)):
        # Append cells to master list
        cellImagesRAW.append(imBW[cells[i]])
        cellImages.append(resizeArray(imBW[cells[i]]))
cellImages = np.array(cellImages)

In [ ]:
fig, ax = plt.subplots(nrows=6, ncols=6, figsize=(12,12))
j = 5
for i in ax.ravel():
    i.imshow(cutArr(cellImages[j]), cmap="gray")
    j = j+1
    i.axis("off")

## Find vacuole locations

Once the cells are isolated from the larger images, the vacuoles in each cell are found by using skimage.morphology.reconstruction(), which finds the pits of an image. The pits are then cleaned by removing small vacuoles/noise, and removing the background of the cell image.

In [ ]:
vacuoleArr = []
maskArr = []
for cell in cellImagesRAW:
    ind, mask = getVacuholes(cell)
    vacuoleArr.append(ind)
    maskArr.append(mask)

In [ ]:
dt = str(datetime.datetime.now().replace(second=0, microsecond=0).isoformat("_"))

fig, ax = plt.subplots(ncols=4,nrows=36,figsize=(15,80))
for j in ax.ravel():
    j.axis("off")
ax[0,0].set_title("Original cell")  
ax[0,1].set_title("Vacuoles removed")
ax[0,2].set_title("Labelled vacuoles")
#plt.tight_layout()

for i in np.arange(36):
    a,b,c = genExample(cellImagesRAW[i])
    
    ax[i,0].imshow(a)
    ax[i,1].imshow(b)
    ax[i,2].imshow(c)
    ax[i,3].imshow(a)
    vacs = ndi.label(c)
    for j in np.arange(len(vacuoleArr[i])):
        avgvx = int(np.mean((vacuoleArr[i][j,0].start,vacuoleArr[i][j,0].stop)))
        avgvy = int(np.mean((vacuoleArr[i][j,1].start,vacuoleArr[i][j,1].stop)))
        ax[i,3].scatter(avgvy,avgvx,s=10,marker='.',c="white")   

## Get an example image with vacuole locations and cells highlighted

Generate an example of where the found cells and corresponding vacuoles are. The cells are labelled 1..n, and the vacuoles are labeled with small white dots. Both the cells and vacuoles are projected onto an RGB image of the slide. A log file containing the cell number, position, size, and number of vacuoles of each cell is outputted. Another log file containing the cell number, position, and size of each vacuole is also outputted.

In [ ]:
dt = str(datetime.datetime.now().replace(second=0, microsecond=0).isoformat("_"))

fig, ax = plt.subplots(ncols=1,nrows=1,figsize=(15,15))
cells = np.array(cells)
r = io.imread(redimg)
g = io.imread(greenimg)
b = io.imread(blueimg)

arr = np.dstack([r, g, b])
img = toimage(arr)
imBright = img.point(lambda p: p * 2) # Make each pixel brighter
plt.imshow(imBright)

#plt.title("Vacuole finder output "+dt+"\n"+redimg)
plt.tight_layout()
ax.axis("off")

cellData = []
vacData = []

for i in np.arange(np.shape(cells)[0]):      
    avgx = int(np.mean((cells[i,1].start,cells[i,1].stop)))
    xst = cells[i,1].start
    avgy = int(np.mean((cells[i,0].start,cells[i,0].stop)))
    yst = cells[i,0].start
    cellSize = abs((cells[i,0].stop-cells[i,0].start)*(cells[i,1].stop-cells[i,1].start))
    noVacs = len(vacuoleArr[i])
    cellData.append([i,int(avgx),int(avgy),int(cellSize),noVacs])
    
    for j in np.arange(len(vacuoleArr[i])):
        avgvx = int(np.mean((vacuoleArr[i][j,1].start,vacuoleArr[i][j,1].stop)))
        avgvy = int(np.mean((vacuoleArr[i][j,0].start,vacuoleArr[i][j,0].stop)))
        vacSize = abs((vacuoleArr[i][j,0].stop-vacuoleArr[i][j,0].start)*(vacuoleArr[i][j,1].stop-vacuoleArr[i][j,1].start))
        vacData.append([i,int(xst+avgvx),int(yst+avgvy),int(vacSize)])
        plt.scatter(xst+avgvx,yst+avgvy,s=10,marker='.',c="white")
        
    plt.annotate(str(i),xy=(avgx,avgy),color="white")

# Outlier detection through a IQR threshold on cell size
cellData = np.array(cellData)
vacData = np.array(vacData)
outlierMask = (cellData[:,3] < loBound) | (cellData[:,3] > hiBound)
outlierData = cellData[outlierMask]
vOutlierMask = np.zeros(vacData.shape[0], dtype=bool)
# Detect and label outliers, and find vacuoles in cells that are outliers
for i in np.arange(np.shape(outlierData)[0]):
    plt.annotate(str(outlierData[i,0]),xy=(outlierData[i,1],outlierData[i,2]),color="blue")
    vOutlierMask = vOutlierMask + np.array(vacData)[:,0] == outlierData[i,0]
    
np.savetxt("./logs/"+dt+"_cellData.csv", cellData[~outlierMask], fmt="%i", delimiter=",", header=redimg+"\ncellNo,xcoord,ycoord,size,noVacuoles")
np.savetxt("./logs/"+dt+"_vacuoleData.csv", vacData[~vOutlierMask], fmt="%i", delimiter=",", header=redimg+"\ncellNo,xcoord,ycoord,size")
plt.savefig("./figures/output/"+dt+"_outputExampleRGB.png")